In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import librosa
import numpy as np
import pandas as pd
import librosa.display
import keras
import IPython.display as ipd
import numpy as np
from tqdm import tqdm


In [2]:
audio_dataset_path=(r'C:\Users\hpmsi\Downloads\output-20220411T164542Z-001\combined')
metadata=pd.read_csv(r"C:\Users\hpmsi\Downloads\output-20220411T164542Z-001\comb_meta\com.csv")
metadata.head()

,file_name,fold,class
0,do001.wav,1,cryandsob
1,do002.wav,1,cryandsob
2,do003.wav,1,cryandsob
3,do004.wav,1,cryandsob
4,do005.wav,1,cryandsob


In [3]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features
    

In [4]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

1421it [01:10, 20.04it/s]


In [5]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-327.12015, 129.83017, -70.920944, -41.29247,...",cryandsob
1,"[-200.72635, 91.268654, -84.3782, 34.001503, -...",cryandsob
2,"[-361.73053, 115.61554, -68.29292, 15.610739, ...",cryandsob
3,"[-349.65146, 137.25008, -59.295837, 28.300783,...",cryandsob
4,"[-433.82852, 111.65828, -27.588154, 6.815115, ...",cryandsob


In [6]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [7]:
y

array(['cryandsob', 'cryandsob', 'cryandsob', ..., 'sad', 'sad', 'sad'],
      dtype='<U9')

In [8]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [9]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [10]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [12]:
### No of classes
num_labels=y.shape[1]

In [13]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [15]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [21]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 250
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/250
19/36 [==============>...............] - ETA: 0s - loss: 0.2228 - accuracy: 0.9095
Epoch 00001: val_loss improved from inf to 0.48847, saving model to saved_models\audio_classification.hdf5
36/36 [==============================] - 0s 4ms/step - loss: 0.2457 - accuracy: 0.8970 - val_loss: 0.4885 - val_accuracy: 0.8211
Epoch 2/250
23/36 [==================>...........] - ETA: 0s - loss: 0.2362 - accuracy: 0.8995
Epoch 00002: val_loss improved from 0.48847 to 0.43222, saving model to saved_models\audio_classification.hdf5
36/36 [==============================] - 0s 4ms/step - loss: 0.2506 - accuracy: 0.8952 - val_loss: 0.4322 - val_accuracy: 0.8386
Epoch 3/250
26/36 [====================>.........] - ETA: 0s - loss: 0.2728 - accuracy: 0.8870
Epoch 00003: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2633 - accuracy: 0.8891 - val_loss: 0.4551 - val_accuracy: 0.8386
Epoch 4/250
27/36 [=====================>........] - ETA: 0

Epoch 30/250
31/36 [========================>.....] - ETA: 0s - loss: 0.2271 - accuracy: 0.8992
Epoch 00030: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2219 - accuracy: 0.9049 - val_loss: 0.4737 - val_accuracy: 0.8246
Epoch 31/250
29/36 [=======================>......] - ETA: 0s - loss: 0.2303 - accuracy: 0.9052
Epoch 00031: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2218 - accuracy: 0.9111 - val_loss: 0.5016 - val_accuracy: 0.8316
Epoch 32/250
30/36 [========================>.....] - ETA: 0s - loss: 0.2437 - accuracy: 0.9052
Epoch 00032: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2328 - accuracy: 0.9093 - val_loss: 0.4441 - val_accuracy: 0.8351
Epoch 33/250
25/36 [===================>..........] - ETA: 0s - loss: 0.2227 - accuracy: 0.8963
Epoch 00033: val_loss did not improve from 0.43222
36/36 [==================

Epoch 60/250
30/36 [========================>.....] - ETA: 0s - loss: 0.1960 - accuracy: 0.9187
Epoch 00060: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1973 - accuracy: 0.9164 - val_loss: 0.5126 - val_accuracy: 0.8386
Epoch 61/250
26/36 [====================>.........] - ETA: 0s - loss: 0.2304 - accuracy: 0.9075
Epoch 00061: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2285 - accuracy: 0.9067 - val_loss: 0.5357 - val_accuracy: 0.8316
Epoch 62/250
31/36 [========================>.....] - ETA: 0s - loss: 0.2165 - accuracy: 0.8931
Epoch 00062: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2285 - accuracy: 0.8944 - val_loss: 0.4457 - val_accuracy: 0.8491
Epoch 63/250
27/36 [=====================>........] - ETA: 0s - loss: 0.2011 - accuracy: 0.9167
Epoch 00063: val_loss did not improve from 0.43222
36/36 [==================

Epoch 90/250
30/36 [========================>.....] - ETA: 0s - loss: 0.1766 - accuracy: 0.9208
Epoch 00090: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1815 - accuracy: 0.9190 - val_loss: 0.5301 - val_accuracy: 0.8316
Epoch 91/250
26/36 [====================>.........] - ETA: 0s - loss: 0.1898 - accuracy: 0.9291
Epoch 00091: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1824 - accuracy: 0.9252 - val_loss: 0.5377 - val_accuracy: 0.8281
Epoch 92/250
27/36 [=====================>........] - ETA: 0s - loss: 0.2215 - accuracy: 0.9039
Epoch 00092: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2071 - accuracy: 0.9093 - val_loss: 0.4830 - val_accuracy: 0.8246
Epoch 93/250
27/36 [=====================>........] - ETA: 0s - loss: 0.2108 - accuracy: 0.9213
Epoch 00093: val_loss did not improve from 0.43222
36/36 [==================

Epoch 120/250
31/36 [========================>.....] - ETA: 0s - loss: 0.1623 - accuracy: 0.9274
Epoch 00120: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1686 - accuracy: 0.9261 - val_loss: 0.5697 - val_accuracy: 0.8035
Epoch 121/250
30/36 [========================>.....] - ETA: 0s - loss: 0.1940 - accuracy: 0.9250 ETA: 0s - loss: 0.2307 - accuracy: 0.90
Epoch 00121: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1981 - accuracy: 0.9243 - val_loss: 0.6332 - val_accuracy: 0.8211
Epoch 122/250
28/36 [======================>.......] - ETA: 0s - loss: 0.1757 - accuracy: 0.9308
Epoch 00122: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1869 - accuracy: 0.9234 - val_loss: 0.6212 - val_accuracy: 0.8035
Epoch 123/250
28/36 [======================>.......] - ETA: 0s - loss: 0.1678 - accuracy: 0.9252
Epoch 00123: val_loss did not im

30/36 [========================>.....] - ETA: 0s - loss: 0.2127 - accuracy: 0.9260
Epoch 00149: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2019 - accuracy: 0.9287 - val_loss: 0.6650 - val_accuracy: 0.8070
Epoch 150/250
29/36 [=======================>......] - ETA: 0s - loss: 0.1768 - accuracy: 0.9246
Epoch 00150: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1782 - accuracy: 0.9252 - val_loss: 0.6383 - val_accuracy: 0.8281
Epoch 151/250
26/36 [====================>.........] - ETA: 0s - loss: 0.2695 - accuracy: 0.9171
Epoch 00151: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.2622 - accuracy: 0.9111 - val_loss: 0.5538 - val_accuracy: 0.8140
Epoch 152/250
28/36 [======================>.......] - ETA: 0s - loss: 0.1785 - accuracy: 0.9286
Epoch 00152: val_loss did not improve from 0.43222
36/36 [============================

Epoch 179/250
27/36 [=====================>........] - ETA: 0s - loss: 0.1652 - accuracy: 0.9271
Epoch 00179: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1679 - accuracy: 0.9287 - val_loss: 0.6151 - val_accuracy: 0.8386
Epoch 180/250
26/36 [====================>.........] - ETA: 0s - loss: 0.2066 - accuracy: 0.9195
Epoch 00180: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1894 - accuracy: 0.9234 - val_loss: 0.5769 - val_accuracy: 0.8421
Epoch 181/250
23/36 [==================>...........] - ETA: 0s - loss: 0.1763 - accuracy: 0.9212
Epoch 00181: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1740 - accuracy: 0.9243 - val_loss: 0.6205 - val_accuracy: 0.8281
Epoch 182/250
30/36 [========================>.....] - ETA: 0s - loss: 0.1806 - accuracy: 0.9271
Epoch 00182: val_loss did not improve from 0.43222
36/36 [==============

Epoch 209/250
29/36 [=======================>......] - ETA: 0s - loss: 0.1699 - accuracy: 0.9256
Epoch 00209: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1673 - accuracy: 0.9278 - val_loss: 0.6487 - val_accuracy: 0.8211
Epoch 210/250
28/36 [======================>.......] - ETA: 0s - loss: 0.1775 - accuracy: 0.9342
Epoch 00210: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1736 - accuracy: 0.9349 - val_loss: 0.6300 - val_accuracy: 0.8211
Epoch 211/250
29/36 [=======================>......] - ETA: 0s - loss: 0.1808 - accuracy: 0.9224
Epoch 00211: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1820 - accuracy: 0.9225 - val_loss: 0.6643 - val_accuracy: 0.8316
Epoch 212/250
26/36 [====================>.........] - ETA: 0s - loss: 0.1671 - accuracy: 0.9279
Epoch 00212: val_loss did not improve from 0.43222
36/36 [==============

Epoch 239/250
26/36 [====================>.........] - ETA: 0s - loss: 0.1589 - accuracy: 0.9351
Epoch 00239: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1618 - accuracy: 0.9313 - val_loss: 0.6215 - val_accuracy: 0.8316
Epoch 240/250
29/36 [=======================>......] - ETA: 0s - loss: 0.1622 - accuracy: 0.9321
Epoch 00240: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1618 - accuracy: 0.9322 - val_loss: 0.6979 - val_accuracy: 0.8281
Epoch 241/250
28/36 [======================>.......] - ETA: 0s - loss: 0.1737 - accuracy: 0.9152
Epoch 00241: val_loss did not improve from 0.43222
36/36 [==============================] - 0s 5ms/step - loss: 0.1650 - accuracy: 0.9190 - val_loss: 0.5875 - val_accuracy: 0.8491
Epoch 242/250
32/36 [=========================>....] - ETA: 0s - loss: 0.1565 - accuracy: 0.9395
Epoch 00242: val_loss did not improve from 0.43222
36/36 [==============

In [17]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8280701637268066


In [18]:
X_test[1]

array([-1.3063849e+02,  1.1165551e+02, -8.7104301e+01,  5.8447609e+01,
       -4.3384937e+01,  2.9110548e+01, -1.6375883e+01,  2.3683420e+01,
       -4.0308213e+00, -7.6600137e+00,  1.0622702e+01, -1.4019006e+01,
        1.2874228e+01, -4.9438267e+00,  8.0033770e+00, -5.4981914e+00,
        9.6272564e+00, -5.0295973e-01, -2.7353075e+00,  6.3974915e+00,
       -3.7514639e+00,  3.1424506e+00, -6.9262648e+00,  6.9378138e-02,
       -5.4487967e+00,  1.1041200e+00, -2.0198722e+00, -2.2589972e+00,
       -4.1758195e-01, -3.9879074e+00, -8.0276453e-01, -1.3649640e+00,
        5.1823225e+00, -1.9565439e+00, -1.1405403e-01, -1.8208673e+00,
       -3.9585388e-01, -5.8239591e-01, -1.7330813e+00, -1.6288793e-01],
      dtype=float32)

In [19]:
model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([5, 6, 5, 5, 6, 1, 5, 1, 1, 0, 4, 3, 0, 2, 6, 5, 2, 1, 3, 5, 5, 1,
       5, 3, 5, 0, 3, 2, 4, 5, 0, 4, 0, 1, 4, 3, 0, 0, 2, 4, 0, 6, 5, 4,
       5, 1, 3, 4, 4, 6, 1, 2, 5, 0, 6, 2, 0, 6, 2, 0, 2, 4, 1, 2, 6, 6,
       6, 1, 0, 4, 6, 6, 6, 5, 4, 6, 0, 3, 0, 6, 5, 3, 4, 4, 0, 3, 2, 6,
       5, 3, 2, 3, 1, 3, 1, 6, 6, 6, 1, 3, 2, 5, 4, 3, 6, 5, 5, 3, 1, 4,
       0, 0, 2, 2, 2, 3, 5, 3, 0, 6, 2, 4, 4, 6, 3, 1, 4, 6, 3, 1, 6, 6,
       4, 2, 0, 6, 1, 4, 0, 3, 3, 0, 2, 0, 5, 4, 0, 5, 5, 6, 6, 1, 5, 0,
       4, 5, 3, 2, 5, 5, 2, 2, 6, 6, 0, 0, 0, 5, 4, 1, 5, 6, 1, 4, 5, 6,
       0, 0, 3, 2, 2, 0, 6, 5, 1, 2, 0, 2, 6, 4, 6, 5, 3, 6, 0, 5, 4, 1,
       4, 0, 6, 5, 0, 6, 4, 6, 6, 6, 5, 6, 5, 5, 0, 0, 1, 4, 1, 6, 2, 5,
       5, 1, 6, 5, 5, 1, 4, 3, 0, 6, 3, 6, 2, 4, 0, 3, 2, 3, 2, 6, 0, 3,
       3, 4, 6, 4, 0, 5, 6, 4, 3, 3, 2, 3, 2, 0, 5, 1, 6, 0, 6, 0, 2, 2,
       2, 1, 5, 5, 3, 1, 3, 5, 0, 5, 6, 5, 1, 1, 2, 6, 6, 4, 6, 6, 3],
      dtype=int64)

In [31]:
filename=r"C:\Users\hpmsi\Downloads\132106__sironboy__woman-scream (1).wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

#print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
#print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
print(prediction_class)

[[-1.6470540e+02  2.6732979e+01 -6.8700127e+01 -5.0641990e+00
  -4.7408385e+00  3.3975830e+01 -2.8795290e+00 -3.2337460e-01
  -2.3452206e+01  2.3330618e+01  1.1420064e+01 -9.9012012e+00
  -1.6797682e+01  1.7326614e+01 -6.9102807e+00 -9.5692414e-01
   3.5794382e+00  2.0893061e+00 -5.4386148e+00  2.2695320e+01
  -1.0479029e+01 -3.1278226e+00  2.3204505e+00  1.7750673e-02
   5.3236084e+00  7.7043641e-01 -3.4205668e+00  3.8611009e+00
  -2.0670660e+00  9.6238403e+00 -1.1486684e+01 -4.0032697e-01
   3.8467505e+00  1.9821206e+00  1.7422614e+00  1.9203607e+00
  -5.8944219e-01 -4.3522793e-01  9.7204828e+00 -5.9988484e+00]]
[6]
['screaming']
